In [1]:
import pandas as pd
import csv
import pymssql
from ast import literal_eval

In [2]:
# Connect to MSSQL database
server = 'localhost'
database = 'BockBluster'
username = 'sa'
password = 'thisIsSuperStrong1234'

connection = pymssql.connect(server=server, database=database, user=username, password=password)
cursor = connection.cursor()

# Define the SQL query
query = "SELECT * FROM publisher"

# Execute the query
cursor.execute(query)

# Fetch all the rows
rows = cursor.fetchall()

# Create an empty list
data_list = []

# Iterate over the rows and extract values into the list
for row in rows:
    value1 = row[0]
    value2 = row[1]
    dictionary = {'id': value1, 'name': value2}
    data_list.append(dictionary)

# Close the cursor and the connection
cursor.close()
connection.close()

In [3]:
df = pd.read_csv('../data/movies.csv')

In [4]:
df['publishers'] = df['publishers'].apply(literal_eval)

df['publishers'] = df['publishers'].apply(lambda x: list(set(x)))

df_new = df[['id', 'publishers']]

df_new = df_new.explode('publishers')

In [5]:
movies_list = df_new.to_dict(orient='records')


mapping = []
for movie in movies_list:
    movie_id = movie['id']
    publisher_name = movie['publishers']
    for publisher in data_list:
        if publisher['name'] == publisher_name:
            publisher_id = publisher['id']
            new_movie = {'movie_id': movie_id, 'publisher_id': publisher_id}
            mapping.append(new_movie)
            break

In [6]:
# Connect to MSSQL database
server = 'localhost'
database = 'BockBluster'
username = 'sa'
password = 'thisIsSuperStrong1234'

connection = pymssql.connect(server=server, database=database, user=username, password=password)

# Insert dictionaries into the table
cursor = connection.cursor()
for dictionary in mapping:
    cursor.execute("INSERT INTO movie_publishers (movie_id, publisher_id) VALUES (%s, %s)", (dictionary['movie_id'], dictionary['publisher_id']))

# Commit the changes and close the connection
connection.commit()
connection.close()